In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip install stable-baselines3[extra]

  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657518 sha256=f2d23ede4ebf00f4267068ad33d6369c50084a346c367345d066e10900395b0a
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\b3\03\54\9141c232861b89be935b37bdde0ea5ab472f5e18fc20623aed
Successfully built gym


In [53]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [21]:
environment_name = "CartPole-v0"

In [22]:
env = gym.make(environment_name)

In [23]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:57.0
Episode:2 Score:28.0
Episode:3 Score:12.0
Episode:4 Score:36.0
Episode:5 Score:19.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [10]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [13]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-8.5794312e-01, -9.1455038e+37,  1.9108307e-01, -1.4090252e+38],
      dtype=float32)

# 3. Train an RL Model

In [37]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log= Log_path)

Using cpu device


In [38]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 219  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 278          |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0074652834 |
|    clip_fraction        | 0.0967       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | -0.00288     |
|    learning_rate        | 0.0003       |
|    loss                 | 5.88         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0157      |
|    value_loss           | 48           |
----------------------------

# 4. Save and Reload Model

In [36]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')
Log_path = os.path.join('Training', 'Logs')

In [18]:
model.save(PPO_path)

In [ ]:
del model

In [27]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [28]:
from stable_baselines3.common.evaluation import evaluate_policy

In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\Admin\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [31]:
env.close()

# 5. Test Model

In [34]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.3282044 ,  0.01333862,  0.09728001, -0.18522004])}]


In [35]:
env.close()

# 6. Viewing Logs in Tensorboard

In [39]:
training_log_path = os.path.join(Log_path, 'PPO_1')

In [41]:
!tensorboard --logdir={training_log_path}

^C


# 7. Adding a callback to the training Stage

In [42]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [43]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [44]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [45]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [46]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [47]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 538  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 409          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0097057875 |
|    clip_fraction        | 0.131        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.685       |
|    explained_variance   | -0.00105     |
|    learning_rate        | 0.0003       |
|    loss                 | 7.02         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0209      |
|    value_loss           | 48.9         |
----------------------------

C:\Users\Admin\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=172.00 +/- 16.15
Episode length: 172.00 +/- 16.15
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 172         |
|    mean_reward          | 172         |
| time/                   |             |
|    total timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008118162 |
|    clip_fraction        | 0.0805      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.61       |
|    explained_variance   | 0.336       |
|    learning_rate        | 0.0003      |
|    loss                 | 22.5        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0192     |
|    value_loss           | 60.5        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 342   |
|    iterations      | 5     |
|    ti

In [48]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [49]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [50]:
env.close()

# 8. Changing Policies

In [51]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
#net_arc = [128,128]

In [52]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [54]:
from stable_baselines3 import DQN

In [55]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [56]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration rate | 0.957    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 629      |
|    time_elapsed     | 0        |
|    total timesteps  | 90       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.919    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 966      |
|    time_elapsed     | 0        |
|    total timesteps  | 171      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.837    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 1563     |
|    time_elapsed     | 0        |
|    total timesteps  | 344      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 2956     |
|    time_elapsed     | 0        |
|    total timesteps  | 2353     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 2974     |
|    time_elapsed     | 0        |
|    total timesteps  | 2459     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 3009     |
|    time_elapsed     | 0        |
|    total timesteps  | 2587     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 3149     |
|    time_elapsed     | 1        |
|    total timesteps  | 4585     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 3142     |
|    time_elapsed     | 1        |
|    total timesteps  | 4646     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 3135     |
|    time_elapsed     | 1        |
|    total timesteps  | 4716     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 3154     |
|    time_elapsed     | 2        |
|    total timesteps  | 6917     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 3154     |
|    time_elapsed     | 2        |
|    total timesteps  | 7003     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 3154     |
|    time_elapsed     | 2        |
|    total timesteps  | 7078     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 3185     |
|    time_elapsed     | 2        |
|    total timesteps  | 9396     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 3190     |
|    time_elapsed     | 2        |
|    total timesteps  | 9484     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 3187     |
|    time_elapsed     | 2        |
|    total timesteps  | 9540     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 2997     |
|    time_elapsed     | 3        |
|    total timesteps  | 11571    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 2996     |
|    time_elapsed     | 3        |
|    total timesteps  | 11630    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 2995     |
|    time_elapsed     | 3        |
|    total timesteps  | 11680    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 3035     |
|    time_elapsed     | 4        |
|    total timesteps  | 13870    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 3042     |
|    time_elapsed     | 4        |
|    total timesteps  | 13990    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 3043     |
|    time_elapsed     | 4        |
|    total timesteps  | 14059    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 3102     |
|    time_elapsed     | 5        |
|    total timesteps  | 16368    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 3099     |
|    time_elapsed     | 5        |
|    total timesteps  | 16437    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 3100     |
|    time_elapsed     | 5        |
|    total timesteps  | 16542    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 3101     |
|    time_elapsed     | 6        |
|    total timesteps  | 18658    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 3096     |
|    time_elapsed     | 6        |
|    total timesteps  | 18788    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 3101     |
|    time_elapsed     | 6        |
|    total timesteps  | 18920    |
----------------------------------
----------------------------------
| rollout/          

In [57]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [58]:
model.save(dqn_path)

In [59]:
model = DQN.load(dqn_path, env=env)

In [60]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(42.1, 14.54957044039445)

In [61]:
env.close()